<a href="https://colab.research.google.com/github/ochi06/portfolio/blob/main/pytorch%EF%BC%BFlightning1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import pytorch_lightning as pl

In [5]:
!pip install pytorch_lightning

In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [7]:
#データの読み込み
from sklearn.datasets import load_iris
iris = load_iris()
x = iris['data']
t = iris['target']

In [8]:
#PyTorchのTensor型へ変換
x = torch.tensor(x, dtype=torch.float32)
t = torch.tensor(t, dtype=torch.int64)

In [9]:
pl.__version__

'2.5.1'

In [10]:
#入力値と目標値をまとめる
dataset = torch.utils.data.TensorDataset(x,t)

In [11]:
#各データのサンプル数を決定
#train : val : test = 60% : 20% : 20%
n_train =int(len(dataset)*0.6)
n_val = int(len(dataset)*0.2)
n_test = len(dataset)-n_train - n_val

In [12]:
#データセットの分割
torch.manual_seed(0)
train, val, test = torch.utils.data.random_split(dataset, [n_train, n_val, n_test])

In [13]:
#バッチサイズ尾定義
batch_size=10

#データローダーを用意
train_loader = torch.utils.data.DataLoader(train, batch_size, shuffle=True, drop_last=True)
val_loader = torch.utils.data.DataLoader(val, batch_size)
test_loader = torch.utils.data.DataLoader(test,batch_size)

ネットワークの定義

In [14]:
class Net(pl.LightningModule):

  def __init__(self):
    super().__init__()
    self.fc1=nn.Linear(4,4)
    self.fc2=nn.Linear(4,3)

  def forward(self,x):
    h=self.fc1(x)
    h=F.relu(h)
    h=self.fc2(h)
    return h

  def training_step(self, batch, batch_idx):
    x,t=batch
    y=self(x)
    loss = F.cross_entropy(y,t)
    return loss

  def configure_optimizers(self):
    optimizer = torch.optim.SGD(self.parameters(),lr=0.01)
    return optimizer

In [15]:
pl.seed_everything(0)
net=Net()

INFO:lightning_fabric.utilities.seed:Seed set to 0


In [16]:
net

Net(
  (fc1): Linear(in_features=4, out_features=4, bias=True)
  (fc2): Linear(in_features=4, out_features=3, bias=True)
)

In [17]:
#学習を行う　Tainer
trainer=pl.Trainer(max_epochs=30,accelerator='gpu',devices=1,deterministic=True)

INFO:pytorch_lightning.utilities.rank_zero:You are using the plain ModelCheckpoint callback. Consider using LitModelCheckpoint which with seamless uploading to Model registry.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [ ]:
#学習の実行
trainer.fit(net, train_loader)

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name | Type   | Params | Mode 
----------------------------------------
0 | fc1  | Linear | 20     | train
1 | fc2  | Linear | 15     | train
----------------------------------------
35        Trainable params
0         Non-trainable params
35        Total params
0.000     Total estimated model params size (MB)
2         Modules in train mode
0         Modules in eval mode
/usr/local/lib/python3.11/dist-packages/pytorch_lightning/loops/fit_loop.py:310: The number of training batches (9) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Training: |          | 0/? [00:00<?, ?it/s]

In [ ]:
#評価指標
x,t=next(iter(test_loader))

In [ ]:
x.shape, t.shape

In [ ]:
#予測値
y=net(x)
y_label=torch.argmax(y, dim=1)

In [ ]:
y_label

In [ ]:
#目標値
t

In [ ]:
#正解率
(t==y_label).sum().float()/len(t)

In [ ]:
from torchmetrics.functional import accuracy

In [ ]:
accuracy(y,t,task='multiclass',num_classes=3)

In [ ]:
class Net(pl.LightningModule):

  def __init__(self):
    super().__init__()
    self.fc1=nn.Linear(4,4)
    self.fc2=nn.Linear(4,3)

  def forward(self,x):
    h=self.fc1(x)
    h=F.relu(h)
    h=self.fc2(h)
    return h

  def training_step(self, batch, batch_idx):
    x,t=batch
    y=self(x)
    loss = F.cross_entropy(y,t)
    #ログを追加
    self.log('train_loss', loss, on_step=True, prog_bar=True)
    self.log('train_acc', accuracy(y,t,task='multiclass',num_classes=3), on_step=True, on_epoch=True, prog_bar=True)
    return loss

  def configure_optimizers(self):
    optimizer = torch.optim.SGD(self.parameters(),lr=0.01)
    return optimizer

In [ ]:
pl.seed_everything(0)

net=Net()
trainer=pl.Trainer(max_epochs=30,accelerator='gpu',devices=1,deterministic=True)
trainer.fit(net, train_loader)

In [ ]:
class Net(pl.LightningModule):

  def __init__(self):
    super().__init__()
    self.fc1=nn.Linear(4,4)
    self.fc2=nn.Linear(4,3)

  def forward(self,x):
    h=self.fc1(x)
    h=F.relu(h)
    h=self.fc2(h)
    return h

  def training_step(self, batch, batch_idx):
    x,t=batch
    y=self(x)
    loss = F.cross_entropy(y,t)
    #ログを追加
    self.log('val_loss', loss, on_step=False, on_epoch=True, prog_bar=True)
    self.log('val_acc', accuracy(y,t,task='multiclass',num_classes=3), on_step=False, on_epoch=True,prog_bar=True)
    return loss

  def test_step(self, batch, batch_idx):
    x,t=batch
    y=self(x)
    loss = F.cross_entropy(y,t)
    #ログを追加
    self.log('test_loss', loss, on_step=False, on_epoch=True)
    self.log('test_acc', accuracy(y,t,task='multiclass',num_classes=3), on_step=False, on_epoch=True)
    return loss

  def validation_step(self, batch, batch_idx):
    x,t=batch
    y=self(x)
    loss = F.cross_entropy(y,t)
    #ログを追加
    self.log('train_loss', loss, on_step=True, prog_bar=True)
    self.log('train_acc', accuracy(y,t,task='multiclass',num_classes=3), on_step=True, on_epoch=True, prog_bar=True)
    return loss

  def configure_optimizers(self):
    optimizer = torch.optim.SGD(self.parameters(),lr=0.01)
    return optimizer

In [ ]:
pl.seed_everything(0)

net=Net()
trainer=pl.Trainer(max_epochs=30,accelerator='gpu',devices=1,deterministic=True)
trainer.fit(net, train_loader, val_loader)

In [ ]:
trainer.callback_metrics

In [ ]:
results=trainer.test(dataloaders=test_loader)

In [ ]:
results